In [42]:
import nltk
import random
from nltk.corpus import movie_reviews
import csv
from nltk.stem.isri import ISRIStemmer

In [43]:
#nltk.download("movie_reviews", 'C:\\Users\\MohamedMaher\\Anaconda3\\nltk_data')

In [44]:
dataset_formatted = []
wordsen = []
wordsar = []
clas = 0

In [45]:
#English Dataset
from stemming.porter2 import stem
for cat in movie_reviews.categories():
    for fileid in movie_reviews.fileids(cat):
        if cat == 'pos':
            clas = 1
        elif cat == 'neg':
            clas = -1
        else:
            clas = 0
        tmp = movie_reviews.words(fileid)
        dataset_formatted.append([tmp, clas])

for word in movie_reviews.words():
    word = stem(word)
    wordsen.extend([word.lower()])

print('After Adding English Patterns: ', len(dataset_formatted))


After Adding English Patterns:  2000


In [29]:
#Arabic Dataset
'''
st = ISRIStemmer()
with open('ArabicDataset2.csv', newline='', encoding='utf-8') as csvfile:
    dataset = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in dataset:
        clas = row[1]
        row = row[0].split()
        for r in row:
                #print('original: ', r)
                #stem = ArListem.light_stem(r)
                #print('stemmed: ',stem)
                r = st.stem(r)
                wordsar.extend([r])
        dataset_formatted.append([row, clas])
        
random.shuffle(dataset_formatted) #Shuffle dataset to avoid biasing towards certain class
print('After Adding Arabic Patterns: ', len(dataset_formatted))
'''

After Adding Arabic Patterns:  10054


In [46]:
words_FD = nltk.FreqDist(wordsen) #take worden frequency
word_featuresen = list(words_FD.keys())[:10000] #take most frequent 10k words

#words_FD = nltk.FreqDist(wordsar) #take wordar frequency
#word_featuresar = list(words_FD.keys())[:10000] #take most frequent 10k words

word_features =  word_featuresen
len(word_features)

10000

In [47]:
word_features

['plot',
 ':',
 'two',
 'teen',
 'coupl',
 'go',
 'to',
 'a',
 'church',
 'parti',
 ',',
 'drink',
 'and',
 'then',
 'drive',
 '.',
 'they',
 'get',
 'into',
 'an',
 'accid',
 'one',
 'of',
 'the',
 'guy',
 'die',
 'but',
 'his',
 'girlfriend',
 'continu',
 'see',
 'him',
 'in',
 'her',
 'life',
 'has',
 'nightmar',
 'what',
 "'",
 's',
 'deal',
 '?',
 'watch',
 'movi',
 '"',
 'sorta',
 'find',
 'out',
 'critiqu',
 'mind',
 '-',
 'fuck',
 'for',
 'generat',
 'that',
 'touch',
 'on',
 'veri',
 'cool',
 'idea',
 'present',
 'it',
 'bad',
 'packag',
 'which',
 'is',
 'make',
 'this',
 'review',
 'even',
 'harder',
 'write',
 'sinc',
 'i',
 'general',
 'applaud',
 'film',
 'attempt',
 'break',
 'mold',
 'mess',
 'with',
 'your',
 'head',
 'such',
 '(',
 'lost',
 'highway',
 '&',
 'memento',
 ')',
 'there',
 'are',
 'good',
 'way',
 'all',
 'type',
 'these',
 'folk',
 'just',
 'didn',
 't',
 'snag',
 'correct',
 'seem',
 'have',
 'taken',
 'pretti',
 'neat',
 'concept',
 'execut',
 'terribl

In [32]:
#Add Arabic Lexicon words
pos_words = []
neg_words = []

with open('NileULex2.csv', newline='', encoding='utf-8') as csvfile:
    lexicon = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in lexicon:
        clas = row[1]
        row = row[0].split()
        if clas == '1':
            for r in row:
                pos_words.extend([r])
        else:
            for r in row:
                neg_words.extend([r])

In [48]:
#Add English Lexicon words
with open('Hu and Liu Lexicon.csv', newline='', encoding='latin-1') as csvfile:
    lexicon = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in lexicon:
        clas = row[1]
        row = row[0].split()
        if clas == '1':
            for r in row:
                pos_words.extend([r.strip()])
        else:
            for r in row:
                neg_words.extend([r.strip()])

In [49]:
#read negation words
negation_words = []
with open('negation_words.txt', encoding='utf-8') as f:
    negation_words = f.read().splitlines() 

In [50]:
#read stop words
with open('stop_words.txt', encoding='utf-8') as f:
    stop_words = f.read().splitlines() 
word_features2 = []
for w in word_features:
    if not w in stop_words:
        word_features2.append(w)
print('Total Word Features: ', len(word_features2))

Total Word Features:  9920


In [51]:
print(len(dataset_formatted))
feature_set = []
feature_set2 = []
class_set = []
j = 0
for (comment, clas) in dataset_formatted:
    j += 1
    words = list(comment)
    feat = {}
    features = []
    for w in word_features2:
        flag = w in words
        if flag:
            features.extend([1])
        else:
            features.extend([0])
        feat[w] = flag
        
    feature_set.append(features)
    feature_set2.append([feat, clas])
    class_set.extend([clas])
print(len(dataset_formatted))

2000
2000


In [52]:
M = len(feature_set)
N = len(feature_set[0])
print(M, ', ', N)

2000 ,  9920


In [53]:
#DNN Classifier
slic = int(M*0.1)
testing_set = feature_set[:slic]
class_testing_set = class_set[:slic]
training_set = feature_set
class_training_set = class_set

In [54]:
import pickle
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5,), random_state=1)
clf.fit(training_set, class_training_set)
output = clf.predict(testing_set)
wrong = sum(i != j for i, j in zip(output, class_testing_set))
acc = (len(output) - wrong)/len(output)
print(wrong, len(output))


1 200


In [55]:
print(acc)
with open('dnn_classifier.pkl', 'wb') as fid:
    pickle.dump(clf, fid, protocol=2) 

0.995


In [ ]:
output
class_set


In [ ]:
M = len(feature_set2)
slic = int(M*0.1)
testing_set2 = feature_set2[:slic]
training_set2 = feature_set2[slic:]

In [ ]:
#NaiveBayes Classifier 
clf = nltk.NaiveBayesClassifier.train(training_set2)
nltk.classify.accuracy(clf, testing_set2)


In [ ]:
#classify a comment
def comm_classifier(comm):
    #comm = 'الراجل ده برنس و بيقول كلام زى الفل'
    comm = 'He is a bad actor and I think he was like a fool in the last movie'
    
    sum_words = 1e-9
    cn_value = 0
    neg = 1
    comm = comm.split()
    for w in comm:
        if w in stop_words:
            continue
        elif w in negation_words:
            neg *= -1
            continue
        if w in pos_words:
            print('POS: ', w)
            cn_value += (neg * 1)
            sum_words += 1
        if w in neg_words:
            print('NEG: ', w)
            cn_value += (neg * -1)
            sum_words += 1
    
    val = cn_value/sum_words
    if val > 0.3:
        val = 1
    elif val < -0.3:
        val = -1
    elif val > -0.1 and val < 0.1:
        val = 0
    #else:
     #   val = clf.predict(comm)
    return val

In [ ]:
comm_classifier('')

In [ ]:
from googletrans import Translator
translator = Translator()
print(translator.translate('dh a7sn brnamg shofto fe 7yate', src='ar', dest='ar').text)
#print(translator.detect('dh a7sn brnamg shofto fe 7eate'))

from langdetect import detect
#print(detect("dh a7sn brnamg shofto fe 7eate"))

In [57]:
import pickle

with open('english_features', 'wb') as fp:
    pickle.dump(word_features2, fp, protocol=2)

In [56]:
len(word_features2)

9920